In [1]:
import json
import os
from PIL import Image
from torchvision import transforms


In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [10]:
# Load JSON data
with open('datasets/data_local.json') as f:
    data = json.load(f)["images"]

In [11]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [12]:
import torch
from transformers import ViTModel, ViTFeatureExtractor

# Load the ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')
model.eval()  # Set the model to evaluation mode

c:\Users\ravik\Ravi\Projects\Gold_ImageSimilarity\enve\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [14]:
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(image).last_hidden_state[:, 0, :]  # Use CLS token
    return features.squeeze().numpy()

# Extract features for all images
features = []
for entry in data:
    feature_vector = extract_features(str(entry['file']))
    features.append(feature_vector)

In [39]:
import faiss
import numpy as np

# Convert features to numpy array
features_np = np.array(features).astype('float32')

# Create a FAISS index
index = faiss.IndexFlatL2(features_np.shape[1])  # L2 distance
index.add(features_np)  # Add feature vectors to the index
# Save the FAISS index
faiss.write_index(index, 'faiss_index.index')

In [40]:
def predict(image_path, k=5):
    print(image_path)
    query_feature = extract_features(image_path)
    query_feature = np.array([query_feature]).astype('float32')
    
    # Search the FAISS index
    distances, indices = index.search(query_feature, k)
    
    
    results = []
    for i, idx in enumerate(indices[0]):
        match_percentage = (1 - distances[0][i] / np.max(distances)) * 100  # Example of distance-based matching
        results.append((data[idx]['label'], data[idx]['product'], match_percentage))
        
    return results



In [41]:
# Example usage
results = predict(r'C:\Users\ravik\Ravi\Projects\Gold_ImageSimilarity\datasets\Test\Rings_Weddings.jpg', k=5)
print(results)
for product, subproduct, percentage in results:
    print(f"Product: {product}, Subproduct: {subproduct}, Match Percentage: {percentage:.2f}%")

C:\Users\ravik\Ravi\Projects\Gold_ImageSimilarity\datasets\Test\Rings_Weddings.jpg
[('Rings', 'Casuals', 15.209394693374634), ('Rings', 'Engagement', 11.55012845993042), ('Rings', 'Engagement', 9.067630767822266), ('Rings', 'Engagement', 3.1344592571258545), ('Rings', 'Engagement', 0.0)]
Product: Rings, Subproduct: Casuals, Match Percentage: 15.21%
Product: Rings, Subproduct: Engagement, Match Percentage: 11.55%
Product: Rings, Subproduct: Engagement, Match Percentage: 9.07%
Product: Rings, Subproduct: Engagement, Match Percentage: 3.13%
Product: Rings, Subproduct: Engagement, Match Percentage: 0.00%
